In [1]:
%matplotlib inline
import numpy as np
import preprocessing as prep
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.utils.model_zoo as model_zoo
from sklearn.metrics import roc_auc_score
import pdb

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [2]:
orc_ARSN        = np.load('dataset/ARSN_oracle.npy', encoding='bytes')
orc_ARSN_label  = np.load('dataset/ARSN_oracle_label.npy', encoding='bytes')
test_ARSN       = np.load('dataset/ARSN_test.npy', encoding='bytes')
test_ARSN_label = np.load('dataset/ARSN_test_label.npy', encoding='bytes')

#train_MESS       = np.load('dataset/MESSIDOR_train.npy', encoding='bytes')
#train_MESS_label = np.load('dataset/MESSIDOR_train_label.npy', encoding='bytes')
#test_MESS        = np.load('dataset/MESSIDOR_test.npy', encoding='bytes')
#test_MESS_label  = np.load('dataset/MESSIDOR_test_label.npy', encoding='bytes')



In [3]:
#Preprocess images
orc_ARSN   = np.transpose(orc_ARSN, (0, 3, 1, 2))
test_ARSN  = np.transpose(test_ARSN, (0, 3, 1, 2))
#train_MESS = np.transpose(train_MESS, (0,3,1,2))
#test_MESS  = np.transpose(test_MESS, (0,3,1,2))
orc_ARSN_prep, test_ARSN_prep   = prep.clean(orc_ARSN,test_ARSN,512)
#train_MESS_prep, test_MESS_prep = prep.clean(train_MESS,test_MESS,512)

In [4]:
# Load inception model
inception = models.inception_v3()

# Inception v3 pretrained weights
model_dir = '/afs/ece.cmu.edu/usr/sadom/Private/Active_learning/pretrained_model'
model_urls = {'inception_v3_google': 
    'https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth',}
inception.load_state_dict(model_zoo.load_url(
    model_urls['inception_v3_google'],model_dir=model_dir))
inception.aux_logits = False

In [5]:
class Medal(nn.Module):
    def __init__(self, pretrain_inception, num_classes):
        super(Medal, self).__init__()
        self.inception = pretrain_inception
        self.linear_0        = nn.Linear(1000, 512)
        self.dropout         = nn.Dropout(0.5)
        self.linear_1        = nn.Linear(512, 128)
        self.linear_2        = nn.Linear(128, num_classes)
        self.relu            = nn.ReLU()
        self.sigmoid         = nn.Sigmoid()
        
        self.init_weight()
        
    def forward(self, images):
        out = self.inception(images)
        out = self.linear_0(out)
        out = self.dropout(out)
        out = self.relu(self.linear_1(out))
        out = self.linear_2(out)
        
        return self.sigmoid(out)
    
    def init_weight(self):
        self.linear_0.weight.data.uniform_(-0.1,0.1)
        self.linear_1.weight.data.uniform_(-0.1,0.1)
        self.linear_2.weight.data.uniform_(-0.1,0.1)
        
        self.linear_0.bias.data.fill_(0)
        self.linear_1.bias.data.fill_(0)
        self.linear_2.bias.data.fill_(0)
        

In [6]:
class MedalTrainer:
    def __init__(self, model, batch_size, max_epochs=1, path='model.pt'):
        global orc_ARSN_prep, orc_ARSN_label, test_ARSN_prep, test_ARSN_label
        # feel free to add any other parameters here
        self.model      = model
        self.batch_size = batch_size
        self.orc_ARSN   = orc_ARSN_prep
        self.orc_ARSN_label  = orc_ARSN_label
        self.test_ARSN       = test_ARSN_prep
        self.test_ARSN_label = test_ARSN_label
        #self.train_MESS       = train_MESS_prep
        #self.train_MESS_label = train_MESS_label
        #self.test_MESS        = test_MESS_prep
        #self.test_MESS_label  = test_MESS_label
        self.train_losses = []
        self.test_losses  = []
        self.test_acc     = []
        self.test_auc_score = []
        self.epochs = 0
        self.prev_epoch = 0
        self.max_epochs = max_epochs
        self.path = path
        
        # TODO: Define your optimizer and criterion here
        self.optimizer = torch.optim.Adam(model.parameters(),lr=0.0001, weight_decay=1e-6)
        self.criterion = nn.BCELoss()

    def train(self):
        self.model.train() # set to training mode
        epoch_loss = 0
        batch_num = 0
        index = np.random.choice(orc_ARSN.shape[0], size=(orc_ARSN.shape[0],), replace=False)
        for i in range(0, self.orc_ARSN.shape[0]-self.batch_size, self.batch_size):
            inputs  = self.orc_ARSN[index[i:i+self.batch_size]]
            targets = self.orc_ARSN_label[index[i:i+self.batch_size]]
            loss = self.train_batch(inputs, targets)
            epoch_loss += loss
            batch_num += 1
            if batch_num % 10 == 0:
                print("At batch",batch_num)
                print("Loss:",loss)
        epoch_loss = epoch_loss / batch_num
        self.epochs += 1
        print('[TRAIN]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs, self.max_epochs, epoch_loss))
        self.train_losses.append(epoch_loss)
        
        return epoch_loss

    def train_batch(self, inputs, targets):
        inputs  = torch.from_numpy(inputs).to(DEVICE).float()
        targets = torch.from_numpy(targets).to(DEVICE).float()
        outputs = torch.flatten(self.model(inputs))
        loss    = self.criterion(outputs,targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()
    
    def test(self):
        self.model.eval()
        test_loss = 0
        auc_score = 0
        test_acc  = 0
        batch_num = 0
        
        with torch.no_grad():
            for i in range(0, self.test_ARSN.shape[0], self.batch_size):
                inputs  = self.test_ARSN[i:i+self.batch_size]
                targets = self.test_ARSN_label[i:i+self.batch_size]
                loss, acc, r_sco = self.test_batch(inputs,targets)
                test_loss += loss
                test_acc  += acc
                auc_score += r_sco
                batch_num += 1
            test_loss /= batch_num
            auc_score  = 100* auc_score/batch_num
            test_acc   = 100* test_acc/(batch_num*self.batch_size)
            
            self.test_losses.append(test_loss)
            self.test_acc.append(test_acc)
            self.test_auc_score.append(auc_score)
            print('[Test]  Epoch [%d/%d]   Loss: %.4f Accuracy: %.4f AUC_Score: %.4f'
                      % (self.epochs, self.max_epochs, test_loss, test_acc, auc_score))
        return test_loss, test_acc, auc_score
            
    def test_batch(self, inputs, targets):
        inputs  = torch.from_numpy(inputs).to(DEVICE).float()
        targets = torch.from_numpy(targets).to(DEVICE).float()
        outputs = torch.flatten(self.model(inputs))
        loss    = self.criterion(outputs,targets) 
        predicted = torch.round(outputs.data)
        correct   = (predicted == targets).sum()
        auc_score = roc_auc_score(targets,outputs.data)
        
        return loss.item(), correct.item(), auc_score
        
    def load(self):
        checkpoint = torch.load(self.path)
        self.prev_epoch = checkpoint['epoch']
        self.model.load_state_dict(checkpoint['state_dict'])
        self.model.cuda()
        print("Model Loaded!")
        
    def save(self):
        self.model.cpu()
        state = {'epoch': self.epochs+self.prev_epoch, 
                 'state_dict': self.model.state_dict()}
        torch.save(state, self.path)
        self.model.cuda()
        print("Model saved!")

In [7]:
BATCH_SIZE = 32
NUM_EPOCHS = 50

load_model = True
path       = 'model/ARSN_baseline.pt'

In [8]:
model   = Medal(inception,1)
model   = nn.DataParallel(model)
model   = model.to(DEVICE)
trainer = MedalTrainer(model,BATCH_SIZE, NUM_EPOCHS, path)

if load_model:
    trainer.load()

Model Loaded!


In [9]:
trainer.test()
pdb.set_trace()
best_nll = 1e30  # set to super large value at first
for epoch in range(NUM_EPOCHS):
    nll = trainer.train()
    if nll < best_nll:
        trainer.save()

[Test]  Epoch [0/50]   Loss: 3.4615 Accuracy: 64.0625 AUC_Score: 52.2731
--Return--
> <ipython-input-9-53c07dcbe34c>(2)<module>()->None
-> pdb.set_trace()
(Pdb) q


BdbQuit: 